1. make sure that the random seeds are working -
2. run the training for "5" epochs and for each set of hyper-parameters run if e.g. 10 times (each time with different random seed) -
3. learn how to plot the "evolution", i.e. training/testing error vs. epochs for mutliple runs  (use https://seaborn.pydata.org/) -
4. multiple lines will corespond to different "hyper-parameters" ("lr" and maybe "momentum" (0, 0.9, 0.99) ) -
5. try to also compare with SignSGD with two versions of SignSGD with momentum -

In [0]:
! nvidia-smi | grep -A 2 -B 2 "N/A"

|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:04:00.0 Off |                    0 |
| N/A   44C    P0    57W / 149W |  11238MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:05:00.0 Off |                    0 |
| N/A   37C    P0    74W / 149W |   1901MiB / 11439MiB |      8%      Default |
+-------------------------------+----------------------+----------------------+
|   2  Tesla K80           Off  | 00000000:83:00.0 Off |                    0 |
| N/A   43C    P0    58W / 149W |  10529MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   3  Tesla K80           Off  | 00000000:84:00.0 Off |                    0 |
| N/A   26C    P8    31W / 149W |     11MiB / 11439MiB |      0%      Default |
+-------------------------------+-------

In [0]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="3"  # which GPU are we using (from 0 to 3)
import torch
torch.set_num_threads(0)
device="cuda:3"

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import numpy as np
import copy
import pickle
from pandas.core.frame import DataFrame
from torch.autograd import Variable
torch.cuda.is_available()

True

In [0]:
#! rm logs/*

In [0]:
#! ls logs/

In [0]:
#device="cpu"   
def computeErrorForWholeDataset(dataset):
    iteration=0
    with torch.no_grad():
        losses=[]
        for i, (images,labels) in enumerate(dataset):
            iteration+=1

            images =Variable(images).to(device)
            labels =Variable(labels).to(device)

            outputs=model(images)
            loss=loss_function(outputs,labels)
            losses.append(loss.item())
    return np.mean(losses)

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        # input_size:28, same_padding=(filter_size-1)/2, 3-1/2=1:padding
        self.cnn1=nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
        # input_size-filter_size +2(padding)/stride + 1 = 28-3+2(1)/1+1=28
        self.batchnorm1=nn.BatchNorm2d(8)
        # output_channel:8, batch(8)
        self.relu=nn.ReLU()
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
        #input_size=28/2=14
        self.cnn2=nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5, stride=1, padding=2)
        # same_padding: (5-1)/2=2:padding_size. 
        self.batchnorm2=nn.BatchNorm2d(32)
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
        # input_size=14/2=7
        # 32x7x7=1568
        self.fc1 =nn.Linear(in_features=1568, out_features=600)
        self.dropout= nn.Dropout(p=0.5)
        self.fc2 =nn.Linear(in_features=600, out_features=10)
    def forward(self,x):
        x =self.cnn1(x)
        x =self.batchnorm1(x)
        x =self.relu(x)
        x =self.maxpool1(x)
        x =self.cnn2(x)
        x =self.batchnorm2(x)
        x =self.relu(x)
        x =self.maxpool2(x)
        x =x.view(-1,1568)
        x =self.fc1(x)
        x =self.relu(x)
        x =self.dropout(x)
        x =self.fc2(x)
        return x
    
    def partial_grad(self, data, labels, loss_function):

        outputs = self.forward(data)
        loss = loss_function(outputs, labels)
        #set grad to 0
        loss.backward() #compute grad
        return loss
    
    def calculate_loss_grad(self, dataset, loss_function, n_samples):

        total_loss = 0.0
        full_grad = 0.0
        for i_grad, data_grad in enumerate(dataset):
            images, labels = data_grad
            images, labels = Variable(images).to(device), Variable(labels).to(device) #wrap data and target into variable
            total_loss += (1./n_samples) * self.partial_grad(images, labels, loss_function)
        
        
        full_grad =np.sum([(para.grad.data.norm(2)**2).item() for para in self.parameters()]) 
        
        return total_loss, (1./n_samples) * np.sqrt(full_grad)
    
    def sarah_backward(self, dataset, loss_function, n_epoch, learning_rate):

        total_loss_epoch = [0 for i in range(n_epoch)]
        grad_norm_epoch = [0 for i in range(n_epoch)]
        for epoch in range(n_epoch):
            for i, (images,labels) in enumerate(train_load):  
                images =Variable(images).to(device)
                labels =Variable(labels).to(device)
                    
            running_loss = 0.0
            previous_net_sgd = copy.deepcopy(self) #update previous_net_sgd
            previous_net_grad = copy.deepcopy(self) #update previous_net_grad

            #Compute full grad
            previous_net_grad.zero_grad() # grad = 0
            total_loss_epoch[epoch], grad_norm_epoch[epoch] = previous_net_grad.calculate_loss_grad(dataset, loss_function, n_samples)
            print(total_loss_epoch[epoch], grad_norm_epoch[epoch])
            #Run over the dataset
            for i_data, data in enumerate(dataset):
                images, labels = data
                images, labels = Variable(images).to(device), Variable(labels).to(device) #wrap data and target into variable
                
                #Compute prev stoc grad
                previous_net_sgd.zero_grad() #grad = 0
                prev_loss = previous_net_sgd.partial_grad(images, labels, loss_function)
                
                #Compute cur stoc grad
                self.zero_grad() #grad = 0
                cur_loss = self.partial_grad(images, labels, loss_function)
                
                #Backward
                for param1, param2, param3 in zip(self.parameters(), previous_net_sgd.parameters(), previous_net_grad.parameters()): 
                    param1.data -= (learning_rate) * (param1.grad.data - param2.grad.data + (1./n_samples) * param3.grad.data)

                # print statistics
                running_loss += cur_loss.item()
                if i_data % 2500 == 2499:    # print every 2500 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i_data + 1, running_loss / 2500))
                    running_loss = 0.0
                
        return total_loss_epoch, grad_norm_epoch

In [0]:
#transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])
train_dataset= datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset= datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True )
train_load=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=1000, shuffle=True)
test_load=torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

n_epoch = 5
learning_rate = 0.01

n_samples = len(train_load)
#print(n_samples)

model=CNN()
CUDA=torch.cuda.is_available()
if CUDA:
    model=model.cuda()
criterion=nn.CrossEntropyLoss()
total_loss_epoch, grad_norm_epoch =model.sarah_backward(train_load,criterion, n_epoch, learning_rate)

(tensor(2.3199, device='cuda:0', grad_fn=<AddBackward0>), 1.1520811483379685)
(tensor(1.5937, device='cuda:0', grad_fn=<AddBackward0>), 1.1434890029119895)
(tensor(0.9384, device='cuda:0', grad_fn=<AddBackward0>), 0.899508545905635)
(tensor(0.6061, device='cuda:0', grad_fn=<AddBackward0>), 0.605894322740242)
(tensor(0.4504, device='cuda:0', grad_fn=<AddBackward0>), 0.4317017899777938)


The following will kill the kernel and will free memory on GPU for others to use

In [0]:
exit(0)